# House Price Estimation

#### Business Problem : 

In [ ]:
# Import the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# Import the dataset 
df = pd.read_csv('House_Pricing.csv')

In [ ]:
df.head()

In [ ]:
df1 = df.drop(columns = ['house_number','street_name','unit_number','zip_code'])
df2 = df1

In [ ]:
df1.head()

In [ ]:
df.describe()

In [ ]:
df1['sale_price'].min()

In [ ]:
df1['sale_price'].max()

In [ ]:
m = df1['sale_price'].mean()
m

In [ ]:
df1['sale_price'] = df1['sale_price'] >= m

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
df1['sale_price'] = le.fit_transform(df1['sale_price'])
df1.head()

In [ ]:
# # Converting Y variable into 2 categories
# house_price = df['sale_price']
# #set up bins
# bin = [664.0,441986.20551249327,22935778.0]
# #use pd.cut function can attribute the values into its specific bins
# category = pd.cut(house_price,bin,labels = ['0','1'])
# category = category.to_frame()
# category.columns = ['range']
# #concatenate and its bin
# df = pd.concat([df,category],axis = 1)

In [ ]:
# missing_columns = df['garage_type']
# missing_columns = missing_columns.replace('none', np.nan)
# modes = df['garage_type'].mode()
# missing_columns = missing_columns.fillna(modes)

In [ ]:
# df1 = df1['garage_type'].replace('none','attached')

In [ ]:
from sklearn.preprocessing import LabelBinarizer
df_one_hot = df1.copy()
lb = LabelBinarizer()
lb_results = lb.fit_transform(df_one_hot['garage_type'])
lb_results_df = pd.DataFrame(lb_results, columns=lb.classes_)
lb_results_df.head()

In [ ]:
final_df = pd.concat([df_one_hot,lb_results_df],axis = 1)

In [ ]:
df_one_hot_en = final_df.copy()
lb_en = LabelBinarizer()
lb_results_en = lb_en.fit_transform(df_one_hot_en['city'])
lb_results_df_en = pd.DataFrame(lb_results_en, columns=lb_en.classes_)
lb_results_df_en.head()

In [ ]:
final_df_en = pd.concat([df_one_hot_en,lb_results_df_en],axis = 1)

In [ ]:
print("Original dimensions :", df.shape)
print("One hot Encoded dimensions :", final_df_en.shape)
final_df_en.head()

In [ ]:
final_df_en.info()

In [ ]:
final_df_en = final_df_en.drop(columns =['garage_type','city'])

In [ ]:
final_df_en.info()

In [ ]:
final_df_en['sale'] = final_df_en['sale_price']
final_df_en.head()

In [ ]:
final_df_en = final_df_en.drop(columns = ['sale_price'])

In [ ]:
final_df_en.head()

In [ ]:
final_df_en = final_df_en.rename(columns = {'sale':'sale_price'})

In [ ]:
final_df_en.head()

In [ ]:
final_df_en['has_fireplace'] = final_df_en['has_fireplace'].astype(int)

In [ ]:
final_df_en['has_pool'] = final_df_en['has_pool'].astype(int)

In [ ]:
final_df_en['has_central_heating'] = final_df_en['has_central_heating'].astype(int)

In [ ]:
final_df_en['has_central_cooling'] = final_df_en['has_central_cooling'].astype(int)

In [ ]:
final_df_en.head()

In [ ]:
final_df_en.dtypes

In [ ]:
array = final_df_en.values
X = array[:,1:63]
y = df['sale_price']

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
final_df_en.head()

In [ ]:
# from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# labelencoder = LabelEncoder()
# X[:,7] = labelencoder.fit_transform(X[:,7])
# onehotencoder = OneHotEncoder(categorical_features = [7])
# X = onehotencoder.fit_transform(X).toarray()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()
regressor.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import make_scorer,mean_squared_error
from sklearn.model_selection import GridSearchCV
# Number of trees in random forest
# n_estimators = [500,1000,3000]
# Number of features to consider at every split
# max_features = [1.0, 0.3,0.1]
# Maximum number of levels in tree
# max_depth = [4,6]
# Minimum number of samples required at each leaf node
# min_samples_leaf = [3,5,9,17]
# Method of selecting samples for training each tree
# learning_rate = [1.0,0.05,0.02,0.01]
param_grid = { 
    'n_estimators': [500,1000,3000],
    'max_features': [1.0,0.3,0.1],
    'max_depth' : [4,6],
    'min_samples_leaf' : [3,5,9,17]
}
grid = GridSearchCV(estimator = regressor,scoring = make_scorer(mean_squared_error), param_grid=param_grid,n_jobs = -1)
grid.fit(x_train,y_train)

#print(grid.grid_scores_)

In [ ]:
best_accuracy = grid.best_score_
best_parameters = grid.best_params_
print(best_accuracy)
print(best_parameters)

In [ ]:
# Predicting the Test set results
y_pred = regressor.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

In [ ]:
# Fitting GBM to the Training set
classifier = GradientBoostingRegressor()
classifier.fit(x_train, y_train)

# Predicting the Test set results
y_pred1 = classifier.predict(x_test)

In [ ]:
param_grid_en = { 
    'n_estimators': [500,1000,3000],
    'max_features': [1.0,0.3,0.1],
    'max_depth' : [4,6],
    'min_samples_leaf' : [3,5,9,17],
    'learning_rate' : [0.1, 0.05, 0.02, 0.01]
}
grid_gb = GridSearchCV(estimator = classifier,scoring = make_scorer(mean_squared_error), param_grid=param_grid_en,n_jobs = -1)
grid_gb.fit(x_train,y_train)